This script is used for preprocessing the corpus:
Input is a xml file with required content between special tags.
Output is a txt file with useful content extracted from xml file:
1. No unicode caracters have been deleted 
2. Unicode string have been normolized
3. Lemmatization done by using treetagger
4. Changing to lowercase

"Ce corpus, compilé automatique- ment (Prokopidis et al., 2012), est constitué de pages Web totalisant environ 50 millions de mots. Le prétraitement appliqué au corpus comprend l’extraction du texte des documents XML (contenant au moins 50 mots), la normalisation des caractères, la lemmatisation au moyen de TreeTagger (Schmid, 1994) et la mise en minuscules. La lemmatisation, qui consiste à remplacer les variantes flexionnelles d’une même forme lexicale par sa forme canonique, a été appliquée parce qu’elle offre parfois une légère amélioration des résultats de l’approche distributionnelle."

In [ ]:
import sys, os, re, argparse, codecs, unicodedata
from xml.dom.minidom import parseString
import treetaggerwrapper as ttw

In [15]:
#replace the no unicode caracters
def replace_charas(string):
    string = re.sub(r"\u0153", "oe", string)
    string = re.sub(r"\u0152", "OE", string)
    string = re.sub(r"[\u2018\u2019]", "'", string)
    string = re.sub(r"[\u201C\u201D]", '"', string)
    return string

In [16]:
# to get a normalized output 
def normalize_charas(string):
    string = unicodedata.normalize('NFC', string)
    string = replace_charas(string)
    string = string.encode('latin-1', errors='ignore')
    string = string.decode('latin-1')
    return string 

In [17]:
#split each line of treetagger's output into three parts
def get_tagger_output(string):
    triple = string.split("\t")
    if len(triple) ==3:
        return triple
    else:
        return None  

In [18]:
def lemmatize(tagger, string):
    sentences = []
    current_sentence = []
    lines = tagger.TagText(string)
    print(lines[2:6])
    for line in lines:
        triple = get_tagger_output(line)
        # for sure that triple is not empty
        if triple:
            word, pos, lemma = triple
            # if a word donot have a lemme form, use his form presented in article as token
            # if not, use lemma form as token
            if lemma == u'<unknown>':
                token = word.lower() # to get lower case
            else:
                token = lemma.lower()
            token = replace_charas(token)
            #if is a punctuation
            if pos == 'SENT':
                #if the setence is not empty
                if len(current_sentence):
                    current_sentence.append(token)
                    sentences.append(current_sentence)
                    current_sentence = []
                else:
                    # The sentence is empty. Ignore the punctuation mark.
                    pass
            else:
                current_sentence.append(token)
    # Make sure that the current sentence is empty, or else add to output. for avoiding the situation
    #that the last sentence has no ponctuation at the end. 
    if len(current_sentence):
        sentences.append(current_sentence)
    return sentences

In [19]:
from os import listdir
from os.path import isfile, join
if __name__ == "__main__":
    path = "/home/wang/Bureau/doct_Yiz/donnees/W0065/test"
    files = [f for f in listdir(path) if isfile(join(path, f))and "xml" in f]
    files_paths = [path+"/"+fp for fp in files]
    #print (files_path[2:6])
    tagger = ttw.TreeTagger(TAGLANG='fr', TAGDIR='/home/wang/Bureau/doct_Yiz/treetagger',
                            TAGINENC='utf-8', TAGOUTENC='utf-8') 
    
    nb_tokens = 0
    for filepath in files_paths:
        with codecs.open(filepath, 'r', encoding='utf-8') as f:
            filetext = f.read()
        xmlfile = parseString(filetext.encode('utf-8'))
        paras = []                     
        processed_paras = []
        #get title
        title_stmt = xmlfile.getElementsByTagName('titleStmt')[0]
        title_node = title_stmt.getElementsByTagName('title')[0].firstChild
        if title_node:
            title = title_node.data
            paras.append(title)
        # Get paragraphs
            for p_elem in xmlfile.getElementsByTagName('p'):
                p_txt = p_elem.firstChild.data
                p_type = p_elem.getAttribute('crawlinfo')
                if p_type not in ['boilerplate', 'ooi-lang', 'ooi-length']:
                    paras.append(p_txt)
        for para in paras:
            para = normalize_charas(para)
            sents = lemmatize(tagger, para)
            processed_paras.append(sents)

        with open("output_test.txt", 'a', encoding='utf-8') as f:
            for para in processed_paras:
                for sent in para:
                    f.write(' '.join(sent)+'\n')
                f.write('\n')


['biologie\tNOM\tbiologie', ')\tPUN\t)', '-\tPUN\t-', 'Wikipédia\tNAM\tWikipédia']
['hypothétique\tADJ\thypothétique', 'de\tPRP\tde', 'tous\tPRO:IND\ttout', 'les\tDET:ART\tle']
[',\tPUN\t,', "l'\tDET:ART\tle", 'évolution\tNOM\tévolution', 'désigne\tVER:pres\tdésigner']
["d'\tPRP\tde", 'évolution\tNOM\tévolution', 'est\tVER:pres\têtre', 'ancienne\tADJ\tancien']
['découverte\tNOM\tdécouverte', 'de\tPRP\tde', 'la\tDET:ART\tle', 'génétique\tNOM\tgénétique']
['est\tVER:pres\têtre', 'causée\tVER:pper\tcauser', ',\tPUN\t,', "d'\tPRP\tde"]
['de\tPRP\tde', "l'\tDET:ART\tle", 'évolution\tNOM\tévolution', 'est\tVER:pres\têtre']
['les\tDET:ART\tle', 'hommes\tNOM\thomme', 'aient\tVER:subp\tavoir', 'cherché\tVER:pper\tchercher']
['Moyen\tNAM\tMoyen', 'Âge\tADJ\tÂge', ',\tPUN\t,', 'les\tDET:ART\tle']
['du\tPRP:det\tdu', 'XVIIIe\tNUM\tXVIIIe', 'siècle\tNOM\tsiècle', ',\tPUN\t,']
["'\tPUN\t'", 'idée\tNOM\tidée', "d'\tPRP\tde", 'évolution\tNOM\tévolution']
['que\tKON\tque', 'les\tDET:ART\tle', 'espèces\

['ici\tADV\tici', 'là\tADV\tlà', ',\tPUN\t,', 'on\tPRO:PER\ton']
['UFC\tNAM\tUFC', 'Que\tNAM\tQue', 'Choisir\tNAM\tChoisir', ',\tPUN\t,']
['(\tPUN\t(', 'inavoué\tADJ\tinavoué', ')\tPUN\t)', 'de\tPRP\tde']
['ceci\tPRO:DEM\tceci', 'voté\tVER:pper\tvoter', ',\tPUN\t,', 'la\tDET:ART\tle']
['électrique\tADJ\télectrique', ',\tPUN\t,', 'déjà\tADV\tdéjà', 'très\tADV\ttrès']
[';\tPUN\t;', 'même\tADV\tmême', 'si\tKON\tsi', 'la\tDET:ART\tle']
['ici\tADV\tici', 'là\tADV\tlà', ',\tPUN\t,', 'on\tPRO:PER\ton']
['livre\tNOM\tlivre', 'publié\tVER:pper\tpublier', 'en\tPRP\ten', '2010\tNUM\t@card@']
['de\tPRP\tde', 'pratiquement\tADV\tpratiquement', 'un\tDET:ART\tun', 'million\tNOM\tmillion']
[',\tPUN\t,', 'concernant\tVER:ppre\tconcerner', "l'\tDET:ART\tle", 'accident\tNOM\taccident']
["qu'\tPRO:REL\tque", 'elle\tPRO:PER\telle', 'donne\tVER:pres\tdonner', 'et\tKON\tet']
[',\tPUN\t,', 'dans\tPRP\tdans', 'chaque\tPRO:IND\tchaque', 'réacteur\tNOM\tréacteur']
['livre\tNOM\tlivre', 'publié\tVER:pper\tpublier

['Mondiale\tADJ\tmondial', 'de\tPRP\tde', "l'\tDET:ART\tle", 'Océan\tNAM\tOcéan']
['Assises\tNOM\tassise', 'de\tPRP\tde', 'la\tDET:ART\tle', 'biodiversité\tNOM\tbiodiversité']
['de\tPRP\tde', 'cette\tPRO:DEM\tce', 'troisième\tNUM\ttroisième', 'session\tNOM\tsession']
['Région\tNOM\trégion', 'Poitou-Charentes\tNAM\tPoitou-Charentes', ',\tPUN\t,', 'engagez\tVER:pres\tengager']
['Semaine\tNAM\tSemaine', 'régionale\tADJ\trégional', 'de\tPRP\tde', "l'\tDET:ART\tle"]
['occasion\tNOM\toccasion', ',\tPUN\t,', 'la\tDET:ART\tle', 'Région\tNOM\trégion']
[',\tPUN\t,', "l'\tDET:ART\tle", 'association\tNOM\tassociation', 'Vienne\tNAM\tVienne']
['Barry\tNAM\tBarry', 'Wells\tNAM\tWells', ',\tPUN\t,', 'sur\tPRP\tsur']
['des\tPRP:det\tdu', 'fougères\tNOM\tfougère', 'et\tKON\tet', 'plantes\tNOM\tplante']
['Nature\tNAM\tNature', 'Environnement\tNAM\tEnvironnement', 'organise\tVER:pres\torganiser', 'cette\tPRO:DEM\tce']
[',\tPUN\t,', 'envoyer\tVER:infi\tenvoyer', '3\tNUM\t@card@', 'photos\tNOM\tphoto']
['I

['caractéristiques\tNOM\tcaractéristique', 'de\tPRP\tde', 'la\tDET:ART\tle', 'collecte\tNOM\tcollecte']
['commerciale\tADJ\tcommercial', ',\tPUN\t,', 'légale\tADJ\tlégal', 'ou\tKON\tou']
['bien\tADV\tbien', ',\tPUN\t,', 'en\tPRP\ten', 'effet\tNOM\teffet']
['près\tADV\tprès', 'de\tPRP\tde', 'Ban\tNAM\tBan', 'Me\tABR\tMe']
['effet\tNOM\teffet', 'de\tPRP\tde', 'cette\tPRO:DEM\tce', 'colonisation\tNOM\tcolonisation']
['environnementaux\tADJ\tenvironnemental', 'de\tPRP\tde', 'la\tDET:ART\tle', 'déforestation\tNOM\tdéforestation']
['couvert\tNOM\tcouvert', 'végétal\tADJ\tvégétal', ',\tPUN\t,', 'la\tDET:ART\tle']
['le\tDET:ART\tle', 'recul\tNOM\trecul', 'du\tPRP:det\tdu', 'couvert\tNOM\tcouvert']
[',\tPUN\t,', 'complexe\tADJ\tcomplexe', 'dans\tPRP\tdans', 'ses\tDET:POS\tson']
['est\tVER:pres\têtre', 'lancé\tVER:pper\tlancer', 'dans\tPRP\tdans', 'une\tDET:ART\tun']
['résulte\tVER:pres\trésulter', 'des\tPRP:det\tdu', 'programmes\tNOM\tprogramme', 'de\tPRP\tde']
['réellement\tADV\tréellement', '

[',\tPUN\t,', 'depuis\tPRP\tdepuis', 'plus\tADV\tplus', 'de\tPRP\tde']
['cherche\tVER:pres\tchercher', 'aussi\tADV\taussi', 'à\tPRP\tà', 'favoriser\tVER:infi\tfavoriser']
['aussi\tADV\taussi', 'que\tKON\tque', 'le\tDET:ART\tle', 'combat\tNOM\tcombat']
['gouvernement\tNOM\tgouvernement', 'cherche\tVER:pres\tchercher', 'à\tPRP\tà', 'exploiter\tVER:infi\texploiter']
['dans\tPRP\tdans', 'cette\tPRO:DEM\tce', 'région\tNOM\trégion', 'est\tVER:pres\têtre']
['les\tDET:ART\tle', 'exploitants\tNOM\texploitant', 'ne\tADV\tne', 'sont\tVER:pres\têtre']
['brésilienne\tADJ\tbrésilien', ',\tPUN\t,', 'la\tDET:ART\tle', 'déforestation\tNOM\tdéforestation']
['La\tDET:ART\tle', 'déforestation\tNOM\tdéforestation', 'et\tKON\tet', "l'\tDET:ART\tle"]
['choisi\tVER:pper\tchoisir', 'de\tPRP\tde', 'nous\tPRO:PER\tnous', 'intéresser\tVER:infi\tintéresser']
[':\tPUN\t:', 'les\tDET:ART\tle', 'îles\tNOM\tîle', 'du\tPRP:det\tdu']
['que\tPRO:REL\tque', 'nous\tPRO:PER\tnous', 'avons\tVER:pres\tavoir', 'étudié\tVER:ppe

['traversé\tVER:pper\ttraverser', 'les\tDET:ART\tle', 'vignobles\tNOM\tvignoble', 'de\tPRP\tde']
[',\tPUN\t,', 'y\tPRO:PER\ty', '´\tNOM\t´', 'a\tVER:pres\tavoir']
['hui\tVER:pper\thui', ',\tPUN\t,', 'on\tPRO:PER\ton', 'a\tVER:pres\tavoir']
['c\tVER:simp\tc', 'est\tVER:pres\têtre', 'tout\tADV\ttout', 'pour\tPRP\tpour']
['est-il\tADJ\test-il', 'passé\tVER:pper\tpasser', 'depuis\tPRP\tdepuis', 'Chilecito\tNAM\tChilecito']
['Ville\tNOM\tville', 'charmante\tADJ\tcharmant', ',\tPUN\t,', 'vivante\tADJ\tvivant']
['le\tDET:ART\tle', 'départ\tNOM\tdépart', 'pour\tPRP\tpour', 'Santiago\tNAM\tSantiago']
['on\tPRO:PER\ton', 'descend\tVER:pres\tdescendre', 'tous\tPRO:IND\ttout', 'du\tPRP:det\tdu']
['on\tPRO:PER\ton', 'passe\tVER:pres\tpasser', 'la\tDET:ART\tle', 'douane\tNOM\tdouane']
['on\tPRO:PER\ton', 'refait\tVER:pres\trefaire', 'la\tDET:ART\tle', 'queue\tNOM\tqueue']
['Les\tDET:ART\tle', 'bagages\tNOM\tbagage', 'et\tKON\tet', 'vélos\tNOM\tvélo']
['on\tPRO:PER\ton', 'passe\tVER:pres\tpasser', 'a

['est\tVER:pres\têtre', 'la\tDET:ART\tle', 'moins\tADV\tmoins', 'expérimentée\tVER:pper\texpérimenter']
['ce\tPRO:DEM\tce', 'que\tPRO:REL\tque', 'laisse\tVER:pres\tlaisser', 'entendre\tVER:infi\tentendre']
['Sites\tNOM\tsite', 'liés\tADJ\tlié', 'et\tKON\tet', 'cités\tNOM\tcité']
['partielle\tADJ\tpartiel', 'ou\tKON\tou', 'totale\tADJ\ttotal', ',\tPUN\t,']
['3B\tNOM\t3B', 'Conseils\tNAM\tConseils', ',\tPUN\t,', 'cabinet\tNOM\tcabinet']
['communiqué\tNOM\tcommuniqué', 'de\tPRP\tde', 'presse\tNOM\tpresse', 'de\tPRP\tde']
['éolien\tADJ\téolien', 'de\tPRP\tde', 'Walney\tNAM\tWalney', 'est\tVER:pres\têtre']
['engagés\tVER:pper\tengager', 'dans\tPRP\tdans', 'la\tDET:ART\tle', 'construction\tNOM\tconstruction']
['de\tPRP\tde', 'la\tDET:ART\tle', 'seconde\tNUM\tsecond', 'phase\tNOM\tphase']
['Sites\tNOM\tsite', 'liés\tVER:pper\tlier', 'et\tKON\tet', 'cités\tNOM\tcité']
['partielle\tADJ\tpartiel', 'ou\tKON\tou', 'totale\tADJ\ttotal', ',\tPUN\t,']
['3B\tNOM\t3B', 'Conseils\tNAM\tConseils', ',\tPU

['épiphytes\tADJ\tépiphyte', 'dans\tPRP\tdans', 'une\tDET:ART\tun', 'forêt\tNOM\tforêt']
['est\tVER:pres\têtre', 'la\tDET:ART\tle', 'diversité\tNOM\tdiversité', 'naturelle\tADJ\tnaturel']
['de\tPRP\tde', 'la\tDET:ART\tle', 'biodiversité\tNOM\tbiodiversité', 'est\tVER:pres\têtre']
['«\tPUN:cit\t«', 'biodiversité\tNOM\tbiodiversité', '»\tPUN:cit\t»', 'est\tVER:pres\têtre']
['de\tPRP\tde', 'la\tDET:ART\tle', 'Terre\tNAM\tTerre', 'de\tPRP\tde']
['sommet\tNOM\tsommet', 'européen\tADJ\teuropéen', 'de\tPRP\tde', 'Göteborg\tNAM\tGöteborg']
['biological\tADJ\tbiological', 'diversity\tNOM\tdiversity', 'a\tVER:pres\tavoir', 'été\tVER:pper\têtre']
[',\tPUN\t,', 'le\tDET:ART\tle', 'terme\tNOM\tterme', 'et\tKON\tet']
['1992\tNUM\t@card@', ',\tPUN\t,', 'le\tDET:ART\tle', 'sommet\tNOM\tsommet']
['variabilité\tNOM\tvariabilité', 'des\tPRP:det\tdu', 'organismes\tNOM\torganisme', 'vivants\tADJ\tvivant']
['la\tDET:ART\tle', 'Convention\tNOM\tconvention', 'sur\tPRP\tsur', 'la\tDET:ART\tle']
['contraction\t